In [ ]:
# imports

import os
import sys
import json
import copy
import importlib
import numpy as np

thisdir = os.getcwd()
topdir = os.path.abspath(os.path.join(thisdir, '../../../'))
sys.path.append(topdir)

from studies.clusters_2024.preprocessing.preprocessor import get_metype
from studies.clusters_2024.nmf.nmf_training import find_files
import studies.clusters_2024.nmf.nmf_testing_pattern as evaltools
importlib.reload(evaltools)

In [ ]:
# era and layer settings

eras = [
    'B-v1',
    'C-v1',
    'D-v1',
    'E-v1',
    'E-v2',
    'F-v1',
    'F-v1-part1',
    'F-v1-part2',
    'F-v1-part3',
    'F-v1-part4',
    'G-v1',
    'G-v1-part1',
    'G-v1-part2',
    'G-v1-part3',
    'G-v1-part4',
    'H-v1',
    'I-v1',
    'I-v2'
]
layers = ['BPix1', 'BPix2', 'BPix3', 'BPix4']

# initialize config
config = {}
config['eras'] = eras
config['layers'] = layers

In [ ]:
# set path to input files

# settings
datadir = '/eos/user/l/llambrec/dialstools-output'
year = '2024'
dataset = 'ZeroBias'
reco = 'PromptReco'
mebase = 'PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_{}'

# find files corresponding to settings
input_files = {}
for era in eras:
    mainera, version = era.split('-', 1)
    input_files[era] = {}
    for layer in layers:
        me = mebase.format(layer[-1])
        f = f'{dataset}-Run{year}{mainera}-{reco}-{version}-DQMIO-{me}.parquet'
        f = os.path.join(datadir, f)
        input_files[era][layer] = [f]

# existence check
missing = []
present = []
for era, values in input_files.items():
    for layer, files in values.items():
        for f in files:
            if not os.path.exists(f): missing.append(f)
            else: present.append(f)
if len(missing) > 0:
    raise Exception(f'The following files do not exist: {missing}')
else:
    print(f'Found {len(present)} files.')
    
# add to config
config['input_files'] = input_files
print(json.dumps(input_files, indent=2))

In [ ]:
# set path to nmf model files

modeldir = 'output_20250604/models'

# set up how to match eras
# options:
# - current era for each input file (e.g. for offline application)
# - previous era for each input file (e.g. for online application)
# - one and the same era for each input file

# use current era
#model_eradict = {era: era for era in eras}

# use previous era
#model_eradict = {}
#for i in range(len(eras)): model_eradict[eras[i]] = eras[i-1]
#model_eradict[eras[0]] = eras[1]

# use a fixed era
model_eradict = {era: 'C-v1' for era in eras}

# set path
nmf_files = {}
for era in eras:
    nmf_files[era] = {}
    model_era = model_eradict[era]
    for layer in layers: nmf_files[era][layer] = os.path.join(modeldir, f'nmf_model_{layer.upper()}_{model_era}.pkl')
    
# existence check
missing = []
for era in eras:
    for layer, f in nmf_files[era].items():
        if not os.path.exists(f): missing.append(f)
if len(missing) > 0:
    raise Exception(f'The following files do not exist: {missing}')
    
# add to config
config['nmf_files'] = nmf_files
print(json.dumps(nmf_files, indent=2))

In [ ]:
# settings for preprocessing

config['preprocessing_global_normalization'] = 'avg'
config['preprocessing_local_normalization'] = 'avg_era_C-v1'

In [ ]:
# settings for filtering

# min entries filter
min_entries_filter = {
    'BPix1': 0.5e6,
    'BPix2': 0.5e6/2,
    'BPix3': 0.5e6/3,
    'BPix4': 0.5e6/4
}

# OMS attribute filters
oms_filter_files = {}
oms_filters = []
for era in eras:
    oms_era = era
    if '-part' in era: oms_era = era.split('-part')[0]
    oms_filter_files[era] = f'/eos/user/l/llambrec/pixelae/studies/clusters_2024/omsdata/omsdata_Run2024{oms_era}.json'
for oms_filter_file in oms_filter_files.values():
    if not os.path.exists(oms_filter_file):
        raise Exception(f'File {oms_filter_file} does not exist.')
oms_filters = [
    ["beams_stable"],
    ["cms_active"],
    ["bpix_ready"],
    ["fpix_ready"],
    ["tibtid_ready"],
    ["tob_ready"],
    ["tecp_ready"],
    ["tecm_ready"],
    ["pileup", '>', 25]
]

# HLT rate filter
hltrate_filter_files = {}
hltrate_filters = []
for era in eras:
    hltrate_era = era
    if '-part' in era: hltrate_era = era.split('-part')[0]
    hltrate_filter_files[era] = f'/eos/user/l/llambrec/pixelae/studies/clusters_2024/omsdata/hltrate_Run2024{hltrate_era}.json'
for hltrate_filter_file in hltrate_filter_files.values():
    if not os.path.exists(hltrate_filter_file):
        raise Exception(f'File {hltrate_filter_file} does not exist.')
hltrate_filters = [
    ["HLT_ZeroBias_v*", '>', 5]
]

# add to config
config['min_entries_filter'] = min_entries_filter
config['oms_filter_files'] = oms_filter_files
config['oms_filters'] = oms_filters
config['hltrate_filter_files'] = hltrate_filter_files
config['hltrate_filters'] = hltrate_filters

In [ ]:
# set path to automask data

do_automasking = False

automask_data_file = None
if do_automasking:
    automask_data_dir = '/eos/user/l/llambrec/pixelae/automasking/data/automask_data'
    automask_data_file = os.path.join(automask_data_dir, f'automask_2024.json')
    if not os.path.exists(automask_data_file):
        raise Exception(f'The automask data file {automask_data_file} does not exist.')
    config['automask_data_file'] = automask_data_file
    
# add to config
config['do_automasking'] = do_automasking
config['automask_data_file'] = automask_data_file

In [ ]:
# set path to loss mask data

do_loss_masking = True
loss_masking_zero_frac_threshold = 0.9

loss_masking_zero_frac_files = None
if do_loss_masking:
    loss_masking_zero_frac_files = {}
    for era in eras:
        loss_mask_era = era
        if '-part' in era: loss_mask_era = era.split('-part')[0]
        loss_masking_zero_frac_files[era] = {}
        for layer in layers:
            zerofrac_file = f'/eos/user/l/llambrec/pixelae/studies/clusters_2024/preprocessing/normdata/zerofrac_Run2024{loss_mask_era}_{get_metype(layer)}.npy'
            if not os.path.exists(zerofrac_file):
                raise Exception(f'The file {zerofrac_file} does not exist.')
            loss_masking_zero_frac_files[era][layer] = zerofrac_file

# add to config
config['do_loss_masking'] = do_loss_masking
config['loss_masking_zero_frac_threshold'] = loss_masking_zero_frac_threshold
config['loss_masking_zero_frac_files'] = loss_masking_zero_frac_files

In [ ]:
# set evaluation parameters

# general
batch_size = 3000
loss_threshold = 0.04

# flagging
flagging_patterns = [np.ones((1,8)), np.ones((2,4))]
flagging_threshold = 1e-3

# cleaning
do_per_layer_cleaning = True
cleaning_patterns = {
    'BPix1': [np.ones((2,16))], # two neighbouring modules
    'BPix2': [np.ones((2,16))], # two neighbouring modules
    'BPix3': [np.ones((2,16))], # two neighbouring modules
    'BPix4': [np.ones((2,16))] # two neighbouring modules
}
cleaning_threshold = 1.5

# add to config
config['batch_size'] = batch_size
config['loss_threshold'] = loss_threshold
config['flagging_patterns'] = [el.tolist() for el in flagging_patterns]
config['flagging_threshold'] = flagging_threshold
config['do_per_layer_cleaning'] = do_per_layer_cleaning
config['cleaning_patterns'] = {key: [el.tolist() for el in val] for key, val in cleaning_patterns.items()}
config['cleaning_threshold'] = cleaning_threshold

In [ ]:
# set directory for this config

outputdir = 'output_test_eraCmodel_lowerlossthreshold_higherpileupthreshold'

if os.path.exists(outputdir):
    msg = f'Output directory {outputdir} already exists.'
    raise Exception(msg)

In [ ]:
# set output file

outputfile = os.path.join(outputdir, 'flagged_lumisections.json')
outputfile = os.path.join('/eos/user/l/llambrec/pixelae/studies/clusters_2024/nmf', outputfile)
# note: cannot use getwcd() to define the directory above,
#       as it seems to give some kind of virtual directory that is ok in the notebook
#       but gives errors when used in the lxplus terminal or condor job.

config['outputfile'] = outputfile

In [ ]:
# make the configuration

split_per_era = True
basename = 'temp_config'

if not os.path.exists(outputdir):
    os.makedirs(outputdir)

if split_per_era:
    for era in eras:
        this_config = copy.deepcopy(config)
        this_config['eras'] = [era]
        this_config['input_files'] = {era: input_files[era]}
        this_config['nmf_files'] = {era: nmf_files[era]}
        this_config['oms_filter_files'] = {era: oms_filter_files[era]}
        this_config['hltrate_filter_files'] = {era: hltrate_filter_files[era]}
        this_config['outputfile'] = outputfile.replace('.json', f'_{era}.json')
        if this_config['loss_masking_zero_frac_files'] is not None:
            this_config['loss_masking_zero_frac_files'] = {era: loss_masking_zero_frac_files[era]}
        configfile = os.path.join(outputdir, f'{basename}_{era}.json')
        with open(configfile, 'w') as f:
            json.dump(this_config, f)

else:
    configfile = os.path.join(outputdir, f'{basename}.json')
    with open(configfile, 'w') as f:
        json.dump(config, f)

In [ ]:
# optional: run the configuration

#flagged_run_numbers, flagged_ls_numbers = evaltools.evaluate(config)

# print flagged lumisections
flagged_run_numbers_unique = np.unique(flagged_run_numbers)
print(f'  Found {len(flagged_run_numbers)} flagged lumisections in {len(flagged_run_numbers_unique)} runs.')
for flagged_run_number, flagged_ls_number in zip(flagged_run_numbers, flagged_ls_numbers):
    print(f'- Run {flagged_run_number}, LS {flagged_ls_number}')